In [26]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import make_scorer,accuracy_score
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['Name', 'Ticket', 'Cabin','SibSp','Parch']
        return self
    def transform(self, X, y=None):
        return X.drop(self.colunasIndesejadas,axis=1)


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

y = train['Survived']
X = train.drop('Survived',axis=1)

t = [('num', SimpleImputer(strategy='median'), ['Pclass', 'Age', 'Fare']), ('cat', SimpleImputer(strategy='most_frequent'), ['Sex', 'Embarked'])]
transformer = ColumnTransformer(transformers=t)


clf1 = RandomForestClassifier()
clf2 = BaggingClassifier()
clf3 = ExtraTreesClassifier()
clf4 = AdaBoostClassifier()
clf5 = DecisionTreeClassifier()
clf6 = LogisticRegression()
clf7 = KNeighborsClassifier()

#eclf = StackingClassifier(estimators=[('rf', clf1), ('rbgf', clf2),('et',clf3),('ab',clf4),('hg',clf5)
#,('knn',clf7)], final_estimator=clf6, n_jobs=-1)


eclf = VotingClassifier(estimators=[('rf', clf1), ('rbgf', clf2),('et',clf3),('ab',clf4),('hg',clf5)
,('lr',clf6),('knn',clf7)], n_jobs=-1)

pipeline = Pipeline([('atributos_desejados', AtributosDesejados()),('transformar', transformer),('encoder', OneHotEncoder(handle_unknown='ignore')),('v',VarianceThreshold()),('selecao_caracteristicas', SelectFromModel(LinearSVC())),('alg',eclf)])      

parametros = {'alg__voting': ['hard', 'soft']}
#parametros = {'alg__stack_method': ['auto']}
           
scoring = {'accuracy': make_scorer(accuracy_score)}
modelo = GridSearchCV(pipeline, parametros, scoring=scoring,refit='accuracy')

scores = cross_validate(modelo, X, y,cv=10, scoring=scoring,return_estimator=True)


print(f"Acurácia: {np.mean(scores['test_accuracy'])}"+"\n\n")    

modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('gender_submission.csv',index=False)


Acurácia: 0.8215480649188514


